# song crawler

In [ ]:
import requests
from bs4 import BeautifulSoup
pages = range(1,1730)
menu_page = 'https://lyricvn.com/tag/viet-nam/page/{}/'
song_pages = set()
for page_idx in pages:
    url = menu_page.format(page_idx)
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    for tag in soup.find_all('a'):
        if 'lyricvn.com/loi-bai-hat' in tag.get('href'):
            song_pages.add(tag.get('href'))
    if page_idx % 100 == 0:
        print('Processed {} pages'.format(page_idx))

In [ ]:
import pandas as pd
list_song_pages = list(song_pages)
file_paths = []
segment_file_paths = []
chunk = 0 
for url_idx,url in enumerate(list_song_pages):
    if url_idx % 100 == 0:
        chunk += 1
    file_name = url.replace('https://lyricvn.com/','').replace('/','')+'.txt'
    file_paths.append(f"raw/chunk_{chunk}/{file_name}")
    segment_file_paths.append(f"segment/chunk_{chunk}/segment_{file_name}")
    

pd_songs = pd.DataFrame(list_song_pages,columns=['song_url'])
pd_songs['file_path'] = file_paths
pd_songs['segment_file_path'] = segment_file_paths
pd_songs.to_csv('data/song_from_lyricvn_com/songs_meta_data.csv',index=False)

In [ ]:
import pandas as pd
import os
loaded_pd_songs = pd.read_csv('data/songs_meta_data.csv')
for idx, (url, file_path) in enumerate(loaded_pd_songs.values):
    abs_file_path = "data/"+file_path
    if os.path.exists(abs_file_path):
        continue
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    os.makedirs(os.path.dirname(abs_file_path), exist_ok=True)
    with open(abs_file_path,'w+') as f:
        for tag in soup.find_all('p'):
            br_tags = tag.find_all('br')
            for br_tag in br_tags:
                br_tag.replace_with("\n")
            content = tag.get_text() +'\n'
            if len(content) > 2:
                f.write(content+'\n')
    if idx %100 == 0:
        print('Processed {} pages'.format(idx))
    

# novel cralwer

In [ ]:

import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

link_truyen:str = 'https://truyen.tangthuvien.vn/doc-truyen/{ten_truyen}/chuong-{chuong}'
so_chuong_truyen = {
    "ta-xac-chet-phong-dang-khong-bi-troi-buoc" : 151,
    "thien-dao-do-thu-quan-convert": 2264,
    "tu-chan-lieu-thien-quan": 3171,
    "xuyen-nhanh-nam-than-co-chu%CC%81t-chay!" : 2652,
    "nu-phu-khong-lan-vao-khoai-xuyen" : 475,
    "dap-noi-ban-sat-di-hoc" : 337,
    "xuyen-thu-tho-he-kho-nu" : 252,
    "nhan-vat-phan-dien-hom-nay-cung-that-ngoan" : 1192,
    "khoai-xuyen-he-thong-nhan-vat-phan-dien-boss-dot-kich!" : 1666,
    "tieu-yeu-the" : 758,
    "su-phu-lai-rot-tuyen" : 704,
    "khoai-xuyen-moi-lan-deu-la-ta-nam-thuong" : 2020,
    "khai-cuc-nu-ma-dau-phu-ta": 1144,
    "than-dao-dan-ton": 4934,
    "cau-tai-yeu-vu-loan-the-tu-tien": 959,
    "so-menh-chi-hoan-tuc-menh-chi-hoan": 207,
    "thuy-nhuong-tha-tu-tien-dich!--ai-bao-han-tu-tien!": 458,
    "moi-ngay-deu-cach-hien-hinh-cang-gan-mot-buoc-moi-thien-do-ly-hien-hinh-canh-can-nhat-bo": 503,
    "cac-nguoi-tu-tien-ta-lam-ruong": 444,
    "ta-co-mot-vien-xuc-xac-van-menh-ma-thuat": 276,
    "di-hoa-do-thi": 2500,
    "my-thuc-cung-ung-thuong": 2734,
    "di-thuong-sinh-vat-kien-van-luc": 1778,
}

base_data = '/home/giangtran/working/bavandangvanba/data/truyen_from_tangthuvien_vn/'
cols = ['ten_truyen','truyen_url','file_path','segment_file_path']
truyen_meta_data = []
for ten_truyen in so_chuong_truyen:
    abs_folder = base_data+ten_truyen+'/'
    os.makedirs(os.path.dirname(abs_folder), exist_ok=True)
    os.makedirs(os.path.dirname(abs_folder+'raw/'), exist_ok=True)
    os.makedirs(os.path.dirname(abs_folder+'segment/'), exist_ok=True)
    for chuong in range(1,so_chuong_truyen[ten_truyen] + 1):
        raw_file_path  = abs_folder+f"raw/chuong_{chuong}"+'.txt'
        segment_file_paths  = abs_folder+f"segment/chuong_{chuong}"+'.txt'
        link = link_truyen.format(ten_truyen=ten_truyen,chuong=chuong)
        truyen_meta_data.append([ten_truyen,link,raw_file_path,segment_file_paths])

        if os.path.exists(raw_file_path) :
            print(f"Skip {chuong} chuong : {ten_truyen}")
            continue
        response = requests.get(link)
        content = response.content
        soup = BeautifulSoup(content, "html.parser")
        
        tags = soup.find_all('div', class_=lambda x: x and x.startswith('box-chap'))
        text= ""
        for tag in tags:
            text += tag.get_text() + " "
        with open(raw_file_path,'w+') as f:
            f.write(text)
    print(f"Done {chuong} chuong : {ten_truyen}")

pd_truyen = pd.DataFrame(truyen_meta_data,columns=cols)
pd_truyen.to_csv('/home/giangtran/working/bavandangvanba/data/truyen_from_tangthuvien_vn/truyen_meta_data.csv',index=False)

In [ ]:
for i in so_chuong_truyen:
    print(i)

In [ ]:
loaded_pd_songs = pd.read_csv('data/song_from_lyricvn_com/songs_meta_data.csv')
for idx, (url, file_path) in enumerate(loaded_pd_songs.values):
    abs_file_path = "data/"+file_path
    content = open(abs_file_path,'r').read()
    break

In [ ]:
print(content)

In [ ]:
so_chuong_truyen = {
    "ta-xac-chet-phong-dang-khong-bi-troi-buoc" : 151,
    "thien-dao-do-thu-quan-convert": 2264,
    "tu-chan-lieu-thien-quan": 3171,
    "xuyen-nhanh-nam-than-co-chu%CC%81t-chay!" : 2652,
    "nu-phu-khong-lan-vao-khoai-xuyen" : 475,
    "dap-noi-ban-sat-di-hoc" : 337,
    "xuyen-thu-tho-he-kho-nu" : 252,
    "nhan-vat-phan-dien-hom-nay-cung-that-ngoan" : 1192,
    "khoai-xuyen-he-thong-nhan-vat-phan-dien-boss-dot-kich!" : 1666,
    "tieu-yeu-the" : 758,
    "su-phu-lai-rot-tuyen" : 704,
    "khoai-xuyen-moi-lan-deu-la-ta-nam-thuong" : 2020,
    "khai-cuc-nu-ma-dau-phu-ta": 1144,
    "cau-tai-yeu-vu-loan-the-tu-tien": 932,
    "than-dao-dan-ton": 4934,
}
sum(map(lambda x: so_chuong_truyen[x], so_chuong_truyen))

# crawl thivien

In [2]:
import os
import sys
import logging
PRJ_BASE ='/mnt/d/personal_working/bavandangvanba'
base_path = PRJ_BASE
sys.path.append(base_path)

In [3]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(os.path.join(PRJ_BASE,'app_metadata/example.db'))

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

sql_query = 'SELECT * FROM van_indexing'

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all results
rows = cursor.fetchall()

# Don't forget to close the cursor and connection when done
cursor.close()
conn.close()

In [4]:
words = set()
for row in rows:
    tmp_words = [word.strip().lower() for word in row[1].split('\n')]
    words.update(tmp_words)
    

In [5]:
len(words)

14448959

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = 'https://www.thivien.net/searchpoem.php?Title={word}&ViewType=1&Page={page}'

if os.path.exists(os.path.join(PRJ_BASE,'tmp/poem_urls.pkl')):
    poem_urls = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/poem_urls.pkl'))
else:
    poem_urls = set()
for word_idx,word in enumerate(words):
    if word_idx % 100 == 0:
        print(f"Processed {word_idx} words")
        pd.to_pickle(poem_urls,os.path.join(PRJ_BASE,'tmp/poem_urls.pkl'))
    for page in range(1, 10):
        response = requests.get(url.format(word=word, page=page), headers=headers)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all elements by their class name
            elements = soup.find_all('h4', class_='list-item-header')
            
            # Check if any elements were found
            if elements:
                for element in elements:
                    # Extract the text from the <a> tag within the <h4> tag
                    link_text = element.get_text(strip=True) if element.a else 'No text'
                    
                    # Extract the href attribute from the <a> tag to get the relative URL
                    link_url = element.a.get('href', 'No URL') if element.a else 'No URL'
                    
                    # Combine the site's base URL with the extracted relative URL, if necessary
                    complete_url = requests.compat.urljoin(url, link_url) if link_url != 'No URL' else 'No URL'

                    poem_urls.add(complete_url)
            else:
                print('No matching elements were found.')
        else:
            print('Failed to retrieve the webpage')

Processed 0 words
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were found.
No matching elements were

KeyboardInterrupt: 

In [8]:
url.format(word=word, page=page)

'https://www.thivien.net/searchpoem.php?Title=giữ trọn thế giới&ViewType=1&Page=4'

In [7]:
poem_urls

set()